In [1]:
1

1

calc total point 
from p4 j03 sumr tablation func cost eachym par

In [2]:
using Distributed
addprocs(8)

8-element Vector{Int64}:
 2
 3
 4
 5
 6
 7
 8
 9

In [3]:
@everywhere using JSON
@everywhere using JLD2
@everywhere using HDF5
@everywhere using Profile
@everywhere using Dates
@everywhere using Tables
@everywhere using DataFrames
@everywhere using DataFramesMeta
@everywhere using CSV
@everywhere using XLSX
@everywhere using BSON
@everywhere using Distributed
@everywhere using FileIO
# using Printf

In [4]:
@everywhere function get_whole_categ(d)
    if haskey(d, "NoCancers") 
        if length(d) ==1
            return "NoCancers", d["NoCancers"]
        else
            delete!(d, "NoCancers")
        end
    end

    ## Here need empyt check!!
    
    res_comb = d |> (x -> keys(x)) |> (x ->collect(x)) |> (x ->sort(x)) |>  (x ->join(x, "_")) 
    res_count = d |> (x -> values(x)) |> (x ->sum(x))
    return res_comb, res_count
end

In [5]:
@everywhere function gen_ymd_dict()
    # f_mst = "../dicts/Jm01_dis_parse/byomei_cancer_20210701.csv"
    # df_mst = @chain CSV.read(f_mst, header=0,types=Dict(4=>String), DataFrame) begin
    #         @transform(:tag = first.(:Column17, 3))
    #     end
    # d_dis2code2 = Dict{String, String}()
    # for r in eachrow(df_mst)
    #     d_dis2code2[r["Column4"]] = r["tag"]
    #     # print(r["Column4"])
    #     # break
    # end


        
    # d_cs = Dict{Vector{String}, Dict{String, Dict}}()
    # d_ids = Dict{String, Dict{Int32, Any}}()
    
    d_ids_each = Dict{String, Any}()
    d_ids_each["first_ym"] = 0
    d_ids_each["gender"] = 0
    d_ids_each["first_cancers"] = []
    d_ids_each["ym_count"] = 0
    d_ids_each["last_ym"] = 0

    d_ym = Dict{Int32, Float32}()
    d_ymd = Dict{Int32, Float32}()
    for y in 2011:2022
        # println(string(y))
        for m in 1:12
            ym = string(y) * lpad(m, 2, "0")
            d_ym[parse(Int32, ym)] = 0.0
            # println(lpad(m, 2, "0"))
            for d in 1:31
                ymd = ym * lpad(d, 2, "0")
                d_ymd[parse(Int32, ymd)] = 0.0
                # println(lpad(d, 2, "0"))
            end
        end
    end

    # f_iy = "../dicts/Jm01_dis_parse/dfmeta_iy_20231122_150053_929635.csv"
    # df_iy = @chain CSV.read(f_iy, header=1,types=Dict("c2"=>String), DataFrame) begin
    #         end
    # d_iy = Dict{String, Dict{Int32, Float32}}()
    # for r in eachrow(df_iy)
    #     d_iy[r["c2"]] = copy(d_ymd)
    # end
    
    
    println("base dicts generated")
    return d_ym, d_ymd
end

In [6]:
dbs = ["s_" * lpad(i, 3, '0') for i in 0:255]
dbs[1:3]

3-element Vector{String}:
 "s_000"
 "s_001"
 "s_002"

In [7]:
# dx = load("../../../storage/d_id1c/id1ns_d_s_001.jld2")["data"]

In [8]:
# dx
# d2 = load("../../../storage/smr_tabl/smr_tbl_cost_s_001.jld2")["data"]

In [9]:
d_ym, d_ymd = gen_ymd_dict()


base dicts generated


(Dict{Int32, Float32}(201212 => 0.0, 201505 => 0.0, 201607 => 0.0, 201103 => 0.0, 201206 => 0.0, 201403 => 0.0, 201503 => 0.0, 201606 => 0.0, 201608 => 0.0, 201303 => 0.0…), Dict{Int32, Float32}(20110131 => 0.0, 20120227 => 0.0, 20120327 => 0.0, 20201020 => 0.0, 20201124 => 0.0, 20130711 => 0.0, 20140112 => 0.0, 20110714 => 0.0, 20190824 => 0.0, 20220716 => 0.0…))

In [10]:
# x = copy(d_ym)
# x

In [11]:
# for (i,(k,v)) in enumerate(dx)
#     println(i)
#     if v == ""
#         print(k)
#         break
#     end
    
# end

In [12]:
# l_whole = []
# l_each = []
# df_whole = DataFrame()
# x = []
for f in dbs[1:2]
    break
# for f in dbs[128:256]
    println(f)
    d_ym, d_ymd = gen_ymd_dict()
    # ../../../storage/smr_tabl/smr_tbl_cs_s_000.jld2
    # "../../../storage/smr_tabl/smr_tbl_cost_s_000.jld2"
    # f_in1 = "../../../storage/smr_tabl/smr_tbl_cs_"*f*".jld2"

    # ../../../storage/d_id1c/id1ns_d_s_196.jld2
    
    # f_in1 = "../../../storage/d_id1c/id1ns_d_"*f*".jld2"
    f_in1 = "../../../storage/d_id1c/id1ns_d_"*f*".jld2"
    # f2 is id1n to ym cost dict
    # f_in2 = "../../../storage/smr_tabl/smr_tbl_cost_"*f*".jld2"
    f_in2 = "../../../storage/smr_tabl/smr_tbl_cost_"*f*".jld2"
    
    f_out_c = "../../../storage/d_id1c/id1ns_smr_cost_"*f*".csv"
    println(f_in1)
    println(f_in2)
    println(f_out_c)
    
    d1 = load(f_in1)["data"]
    #f1 is id1n to cancer categ dict
    #d1: key:id1n, cancer categ
    # println(d1)
    
    d2 = load(f_in2)["data"]
    #d2: key:id1n, value = dict(ym:cost)
    # println(d2)
    d_cancer_ym_cost = Dict{String, Dict}()
    d_cancer_ym_count = Dict{String, Dict}()
    e = Dict{String, Any}()
    # break
    for (k,d_ym_cost) in d2
        cancer_categ = d1[k]
        if !haskey(d_cancer_ym_cost, cancer_categ)
            d_cancer_ym_cost[cancer_categ] = copy(d_ym)
            d_cancer_ym_count[cancer_categ] = copy(d_ym)
        end        
        
        # v2 = d2[k]
        for (ym, cost) in d_ym_cost
            d_cancer_ym_cost[cancer_categ][ym] += cost
            d_cancer_ym_count[cancer_categ][ym] += 1
        #     e2 = copy(e)
        #     e2[""] = k
        #     e2["ym"] = ym
        #     e2["cost"] = cost
        # # push!(df_whole, e)
        #     push!(l_each, e2)
        # # push!(l_whole, e)
        end
        # d_id2c[k] = main_dis
        # l_whole.append(e)
        # break
    end

    l_each = []
    e = Dict{String, Any}()
    for (cancer, d_ym_cost) in d_cancer_ym_cost
        for (ym, cost) in d_ym_cost
            e2 = copy(e)
            e2["cost"] = cost
            e2["ym"] = ym
            e2["cancer"] = cancer
            e2["db"] = f
            e2["count"] = d_cancer_ym_count[cancer][ym]
            push!(l_each, e2)
        end
    end
    
    df_i = DataFrame(l_each)
    CSV.write(f_out_c, df_i)
    # l_each = []
    # save(f_out,"data", d_id2c)
    # # break
    # return 0
end


In [13]:
# d1

In [14]:
# d2

In [15]:
@everywhere function smr_smr_for_cost(f)
    println(f)
    d_ym, d_ymd = gen_ymd_dict()
    f_in1 = "../../../storage/d_id1c/id1ns_d_"*f*".jld2"
    f_in2 = "../../../storage/smr_tabl/smr_tbl_cost_"*f*".jld2"
    f_out_c = "../../../storage/d_id1c/id1ns_smr_cost_"*f*".csv"
    println(f_in1)
    println(f_in2)
    println(f_out_c)
    
    d1 = load(f_in1)["data"]
    d2 = load(f_in2)["data"]
    #d2: key:id1n, value = dict(ym:cost)
    # println(d2)
    d_cancer_ym_cost = Dict{String, Dict}()
    d_cancer_ym_count = Dict{String, Dict}()
    e = Dict{String, Any}()
    # break
    for (k,d_ym_cost) in d2
        cancer_categ = d1[k]
        if !haskey(d_cancer_ym_cost, cancer_categ)
            d_cancer_ym_cost[cancer_categ] = copy(d_ym)
            d_cancer_ym_count[cancer_categ] = copy(d_ym)
        end        
        
        # v2 = d2[k]
        for (ym, cost) in d_ym_cost
            d_cancer_ym_cost[cancer_categ][ym] += cost
            d_cancer_ym_count[cancer_categ][ym] += 1
        end
    end

    l_each = []
    e = Dict{String, Any}()
    for (cancer, d_ym_cost) in d_cancer_ym_cost
        for (ym, cost) in d_ym_cost
            e2 = deepcopy(e)
            e2["cost"] = deepcost
            e2["ym"] = ym
            e2["cancer"] = cancer
            e2["db"] = f
            e2["count"] = d_cancer_ym_count[cancer][ym]
            push!(l_each, e2)
        end
    end
    
    df_i = DataFrame(l_each)
    CSV.write(f_out_c, df_i)
    return 0
end

In [ ]:
results = @distributed (+) for dbi in dbs
    # [250:256]
    # get_d(dbxs)
    smr_smr_for_cost(dbi)
    
    # println(dbxs)
end

      From worker 4:	s_064
      From worker 4:	base dicts generated
      From worker 4:	../../../storage/d_id1c/id1ns_d_s_064.jld2
      From worker 4:	../../../storage/smr_tabl/smr_tbl_cost_s_064.jld2
      From worker 4:	../../../storage/d_id1c/id1ns_smr_cost_s_064.csv
      From worker 3:	s_032
      From worker 3:	base dicts generated
      From worker 3:	../../../storage/d_id1c/id1ns_d_s_032.jld2
      From worker 3:	../../../storage/smr_tabl/smr_tbl_cost_s_032.jld2
      From worker 3:	../../../storage/d_id1c/id1ns_smr_cost_s_032.csv
      From worker 6:	s_128
      From worker 6:	base dicts generated
      From worker 6:	../../../storage/d_id1c/id1ns_d_s_128.jld2
      From worker 6:	../../../storage/smr_tabl/smr_tbl_cost_s_128.jld2
      From worker 6:	../../../storage/d_id1c/id1ns_smr_cost_s_128.csv
      From worker 2:	s_000
      From worker 2:	base dicts generated
      From worker 2:	../../../storage/d_id1c/id1ns_d_s_000.jld2
      From worker 2:	../../../storage/smr_tab

In [ ]:
results

In [ ]:
kokomade